# Title
Description

### Step 0: Function that checks if libraries are imported for different environments

In [1]:
# Import the necessary libraries
import importlib  # Allows for runtime importing of modules
import subprocess  # Allows for the execution of bash commands within Python

# Define the function that checks the modules' installation status and import them
def check_installed_and_import(import_dict):
    # Iterate over the dictionary items
    for module, imported_as in import_dict.items():
        try:
            # Attempt to import the module using importlib
            imported_module = importlib.import_module(module)
            # If the import is successful, add the module to the global namespace under the 'imported_as' alias
            globals()[imported_as] = imported_module
            # Print a success message
            print(f"{module} imported as {imported_as}")
        # If the import fails, catch the exception
        except Exception as e:
            # Print a failure message
            print(f"FAILED: {module} imported as {imported_as}. Trying to install...")
            try:
                # Try to install the module using pip via the subprocess module
                base_module = module.split(".")[0]  # Get the base module, in case we're trying to import a submodule
                subprocess.check_call(['pip', 'install', base_module])
                # Try to import the module again
                imported_module = importlib.import_module(module)
                # If the import is successful this time, add the module to the global namespace under the 'imported_as' alias
                globals()[imported_as] = imported_module
                # Print a success message
                print(f"{module} imported as {imported_as}")
            # If the import still fails, catch the exception
            except Exception as e:
                # Print a final failure message along with the exception message
                print(f"FAILED: {module} imported as {imported_as}. Error: {e}")

### Step 1:  Import necessary libraries

In [2]:
# Call the function with our dictionary to check the modules' installation and import them
check_installed_and_import({
    'os': 'os',
    'numpy': 'np',
    'tensorflow': 'tf',
    'tensorflow.keras': 'keras',
    'tensorflow.keras.layers': 'layers',
    'tensorflow.keras.models': 'models',
    'tensorflow.keras.preprocessing.image': 'image',
    'tensorflow.keras.callbacks': 'callbacks',
    'tensorflow.keras.optimizers': 'optimizers',
})

os imported as os
numpy imported as np


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

tensorflow imported as tf
tensorflow.keras imported as keras
tensorflow.keras.layers imported as layers
tensorflow.keras.models imported as models
tensorflow.keras.preprocessing.image imported as image
tensorflow.keras.callbacks imported as callbacks
tensorflow.keras.optimizers imported as optimizers


### Step 2: Data Preprocessing

In [3]:
# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

# Normalize the data (scale pixel values between 0 and 1)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoded vectors
num_classes = 100
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Data augmentation
data_gen = image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
data_gen.fit(x_train)

169001437/169001437 [==============================] - 6s 0us/step


### Step 3: Build the CNN Architecture

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.4))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))


### Step 4: Compile the Model

In [5]:
opt = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

### Step 5: Training the Model

In [6]:
epochs = 50
batch_size = 64
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(
    data_gen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(x_train) / batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)

Epoch 1/50
781/781 [==============================] - 759s 966ms/step - loss: 4.0695 - accuracy: 0.1049 - val_loss: 3.4209 - val_accuracy: 0.1843
Epoch 2/50
781/781 [==============================] - 751s 962ms/step - loss: 3.1973 - accuracy: 0.2191 - val_loss: 3.2780 - val_accuracy: 0.2158
Epoch 3/50
781/781 [==============================] - 755s 966ms/step - loss: 2.7418 - accuracy: 0.3024 - val_loss: 2.7379 - val_accuracy: 0.3112
Epoch 4/50
781/781 [==============================] - 752s 963ms/step - loss: 2.4956 - accuracy: 0.3523 - val_loss: 2.2584 - val_accuracy: 0.4085
Epoch 5/50
781/781 [==============================] - 752s 962ms/step - loss: 2.3029 - accuracy: 0.3919 - val_loss: 2.0926 - val_accuracy: 0.4491
Epoch 6/50
781/781 [==============================] - 752s 963ms/step - loss: 2.1743 - accuracy: 0.4215 - val_loss: 2.1358 - val_accuracy: 0.4385
Epoch 7/50
781/781 [==============================] - 755s 966ms/step - loss: 2.0608 - accuracy: 0.4472 - val_loss: 2.1340 -

### Step 6: Evaluate the Model

In [7]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

313/313 [==============================] - 34s 108ms/step - loss: 1.2615 - accuracy: 0.6533
Test Accuracy: 65.33%


### Step 7: Fine-tuning